In [1]:
from tensorflow.keras.preprocessing import image
import numpy as np
import sys
import os

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet import MobileNet

from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, GRU
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.layers import TimeDistributed,Bidirectional,GlobalAveragePooling2D
from tensorflow.keras.layers import LSTM,GRU,SimpleRNN,BatchNormalization
from tensorflow.keras import backend as K 
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split as sp
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score


2024-04-30 15:47:26.204946: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-30 15:47:26.206227: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-30 15:47:26.248569: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-30 15:47:26.250004: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


2024-04-30 15:47:29.633357: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-30 15:47:29.637643: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
import tensorflow as tf

tensor = tf.constant([])
print(tensor.device)

/job:localhost/replica:0/task:0/device:CPU:0


In [29]:
img_height = 128
img_width = 128
output = 3

In [30]:

def  get_images(path): 
    image_list = []
    class_list = []    
    for dirname in os.listdir(path):
    # print path to all subdirectories first.
        new_path =os.path.join(path,dirname)
        for dirname, dirnames1,filenames in os.walk(new_path):
            for filename in filenames:
                img = image_to_vector(os.path.join(new_path,filename))
                image_list.append(img)
                class_list.append(dirname)
    return np.array(image_list),class_list

In [31]:
def image_to_vector(img_file_path):

    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_file_path, target_size=(img_height, img_width))
    # convert PIL.Image.Image type to 3D tensor with shape (120, 160, 3)
    x = image.img_to_array(img)
    #y = x.reshape(img_height*img_width*3,1)
    #y= y[0 : img_height*img_width]
    #y= y[2*img_height*img_width : 3*img_height*img_width]
    #y = y.reshape(img_height,img_width)
    return x

## Models

In [32]:
def create_cnn_model():
    model = Sequential() 
   
    model.add(Conv2D(12, (3, 3), input_shape = (img_height,img_width,3) )) 
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))  
    model.add(MaxPooling2D(pool_size =(2, 2))) 
   
    #model.add(TimeDistributed(Flatten()))
    model.add(Flatten())
    model.add(Dense(units=100))

    model.add(Dropout(0.2)) 
    model.add(Dense(output)) 
    model.add(Activation('softmax')) 

    model.compile(loss ='categorical_crossentropy', 
                     optimizer ='adam', 
                   metrics =['accuracy']) 
    return model

In [33]:
def create_cnn_lstm_model():
    model = Sequential() 
   
    model.add(Conv2D(12, (3, 3), input_shape = (img_height,img_width,3) )) 
    model.add(Activation('relu'))
    #model.add(Dropout(0.2))  
    model.add(MaxPooling2D(pool_size =(2, 2))) 
   
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(units=100))
    #model.add(Bidirectional(LSTM(units=100)))

    model.add(Dropout(0.2)) 
    model.add(Dense(output)) 
    model.add(Activation('softmax')) 

    model.compile(loss ='categorical_crossentropy', 
                     optimizer ='adam', 
                   metrics =['accuracy']) 
    return model

In [34]:

def create_cnn_model1():
    model = Sequential() 
    
    model.add(Conv2D(32, (3, 3), input_shape = (img_height,img_width,3),kernel_initializer='VarianceScaling'))
    model.add(Conv2D(32, (3, 3),kernel_initializer='VarianceScaling' ))  
    model.add(BatchNormalization())
    model.add(Activation('relu')) 
    model.add(MaxPooling2D(pool_size =(2, 2))) 
    
    model.add(Conv2D(64, (3, 3),kernel_initializer='VarianceScaling' ))
    model.add(Conv2D(64, (3, 3),kernel_initializer='VarianceScaling' )) 
    model.add(BatchNormalization())
    model.add(Activation('relu')) 
    model.add(MaxPooling2D(pool_size =(2, 2))) 
    
    model.add(Conv2D(128, (3, 3),kernel_initializer='VarianceScaling' )) 
    model.add(Conv2D(128, (3, 3),kernel_initializer='VarianceScaling' )) 
    model.add(BatchNormalization())
    model.add(Activation('relu')) 
    model.add(MaxPooling2D(pool_size =(2, 2))) 
   
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.5)) 
    
    #model.add(TimeDistributed(Flatten()))
    model.add(Flatten())
    #model.add(LSTM(units=100,kernel_initializer='VarianceScaling'))
    #model.add(Bidirectional(LSTM(units=100,kernel_initializer='VarianceScaling')))
    model.add(Dense(units=100, activation='sigmoid'))


    model.add(Dense(output)) 
    model.add(Activation('softmax')) 

    model.compile(loss ='mae', 
                     optimizer ='RMSprop', 
                   metrics =['accuracy']) 
    return model



In [35]:
def pretrained_Xception():
    base_model = Xception(weights="imagenet", include_top=False, input_shape = (img_height,img_width,3))
    base_model.trainable = True ## Not trainable weights
    flatten_layer = Flatten()
    dense_layer_1 = Dense(256, activation='relu')
    #dense_layer_2 = Dense(20, activation='relu')
    prediction_layer = Dense(output, activation='softmax')
    
    model = Sequential([
      base_model,
      flatten_layer,
      dense_layer_1,
      #dense_layer_2,
      prediction_layer
      ])
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy', #değiştirrr
        metrics=['accuracy'],
        )
    
    return model

In [36]:
def pretrained_InceptionV3():
    base_model = InceptionV3(weights="imagenet", include_top=False, input_shape = (img_height,img_width,3))
    base_model.trainable = True ## Not trainable weights
    flatten_layer = Flatten()
    dense_layer_1 = Dense(256, activation='relu')
    #dense_layer_2 = Dense(20, activation='relu')
    prediction_layer = Dense(output, activation='softmax')
    
    model = Sequential([
      base_model,
      flatten_layer,
      dense_layer_1,
      #dense_layer_2,
      prediction_layer
      ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        )
    
    return model

In [37]:
def  pretrained_MobileNet():
    base_model = MobileNet(weights="imagenet", include_top=False, input_shape = (img_height,img_width,3))
    base_model.trainable = True ## Not trainable weights
    flatten_layer = Flatten()
    dense_layer_1 = Dense(256, activation='relu')
    #dense_layer_2 = Dense(20, activation='relu')
    prediction_layer = Dense(output, activation='softmax')
    
    model = Sequential([
      base_model,
      flatten_layer,
      dense_layer_1,
      #dense_layer_2,
      prediction_layer
      ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        )
    
    return model

In [38]:
def pretrained_vgg16():
    base_model = VGG16(weights="imagenet", include_top=False, input_shape = (img_height,img_width,3))
    base_model.trainable = False ## Not trainable weights
    flatten_layer = Flatten()
    dense_layer_1 = Dense(256, activation='relu')
    prediction_layer = Dense(output, activation='softmax')
    
    model = Sequential([
      base_model,
      flatten_layer,
      dense_layer_1,
      #dense_layer_2,
      prediction_layer
      ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        )
    
    return model

In [39]:
def pretrained_resnet():
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape = (img_height,img_width,3))
    base_model.trainable = True ## Not trainable weights
    flatten_layer = Flatten()
    dense_layer_1 = Dense(256, activation='relu')
    #dense_layer_2 = Dense(20, activation='relu')
    prediction_layer = Dense(output, activation='softmax')
    
    model = Sequential([
      base_model,
      flatten_layer,
      dense_layer_1,
      #dense_layer_2,
      prediction_layer
      ])
    
    model.compile(
        optimizer='adam',
        loss='categorical_crossentropy',
        metrics=['accuracy'],
        )
    
    return model

In [40]:
def create_lstm_model():
    model = Sequential() 
   
    #model.add(LSTM(units=100)) 
    model.add(Bidirectional(LSTM(units=100)))

    model.add(Dropout(0.2)) 
    model.add(Dense(output)) 
    model.add(Activation('softmax')) 

    model.compile(loss ='categorical_crossentropy', 
                     optimizer ='adam', 
                   metrics =['accuracy']) 
    return model

In [41]:
def create_checkpoint():
    filepath = "best.hdf5"
    checkpoint = ModelCheckpoint(filepath,
                            monitor='val_loss',
                            verbose=2,
                            save_best_only=True,
                            mode='min')
    return checkpoint

In [42]:
filePath = '/home/muhlabws4/Masaüstü/tkd_data/orj'

In [43]:
features,classes = get_images(filePath)

In [44]:
features = features / 255

In [45]:
encoder = LabelEncoder()
classes = encoder.fit_transform(classes)

In [46]:
rnd_seed = 116
np.random.seed(117)
kf =StratifiedKFold(n_splits=3, random_state=rnd_seed, shuffle =True)

In [47]:
scores = []
scores_auc = []
confussionMatrix = []
total = np.zeros((output,output))
filepath = "best.hdf5"

In [48]:
def cross_validation(model, X, y, n_splits=5, epochs=5, batch_size=32):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    #model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    #result = []
    for train_inx, val_inx in kfold.split(X):
        X_train, X_val = X[train_inx], X[val_inx]
        y_train, y_val = y[train_inx], y[val_inx]
        
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        a = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=2, class_weight={0:7,1:10,2:22})
        #model.fit(X_train, y_train, epochs=epochs)
        #result.append(a.history['val_accuracy'])
        
    #return result

In [50]:

for train_index, test_index in kf.split(features,classes): 
    #result = next(kf.split(df), None)
    classes1 = to_categorical(classes)
    x_train_val =features[train_index]
    x_test = features[test_index]
    y_train_val = classes1[train_index]
    y_test = classes1[test_index]
    
    x_train,x_val,y_train,y_val = sp(x_train_val, y_train_val,random_state=5, test_size=.2)
    
    #model = create_lstm_model()
    #model = create_cnn_model1()
    #model = pretrained_resnet()
    #model = pretrained_vgg16()
    model = pretrained_Xception()
    #model = pretrained_InceptionV3()
    #model = pretrained_MobileNet()
    #model = pretrained_vgg16()
    #model = create_cnn_lstm_model()
    checkpoint = create_checkpoint()
    print('Train... Model')
    model.fit(x_train , y_train, epochs=3, batch_size=10, validation_data=(x_val,y_val),verbose=2,callbacks=[checkpoint])
    model.load_weights("best.hdf5")
    y_pred = model.predict(x_test)
    

    conf_mat = confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
    
    scores_auc.append(roc_auc_score(y_test, y_pred.round(),multi_class="ovr"))
    scores.append(accuracy_score(y_test.argmax(axis=1), y_pred.argmax(axis=1)))
    total = total + conf_mat
    confussionMatrix.append(conf_mat)


Train... Model
Epoch 1/3


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "<frozen runpy>", line 198, in _run_module_as_main
    File "<frozen runpy>", line 88, in _run_code
    File "/home/muhlabws4/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/muhlabws4/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance
      app.start()
    File "/home/muhlabws4/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 736, in start
      self.io_loop.start()
    File "/home/muhlabws4/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/home/muhlabws4/anaconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever
      sel

In [ ]:
print('Scores from each Iteration: ', scores)
print('Average K-Fold Score :' , np.mean(scores))


In [ ]:
print(conf_mat)